## Bridging combat messages spreadsheet to JSON 

In [1]:
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from rdflib.namespace import XSD, RDF, OWL, RDFS
from rdflib import Graph, URIRef, Namespace, BNode, Literal
import owlrl
import json
import re
import shortuuid 

In [2]:
in_file = 'Bridging_Combat_Messages_Breakdown_v2.xlsx'
out_file = 'Bridging2.ttl'

In [3]:
# namespaces
cm = "http://purl.org/artiamas/cm/"
CM = Namespace(cm)

In [4]:
file = load_workbook(filename = in_file)
sheet = file.active

In [5]:
# read the spreadsheet data into a JSON-like python structure
all_data = []

last_column = len(list(sheet.columns))
last_row = len(list(sheet.rows))

for row in range(2, last_row + 1):
    data = {}
    data['sequence'] = row-1
    for column in range(2, last_column + 1):
        col = get_column_letter(column)
        property = sheet[col + str(1)].value
        value = sheet[col + str(row)].value
        if row > 1:
            data[property] = value
    all_data.append(data)

In [6]:
# custom BBode function adds a prefix to a short uuid sequence
def bnode(prefix = ''):
    if prefix:
        return BNode(prefix + '_' + shortuuid.uuid()[:5])
    else:
        return BNode(shortuuid.uuid()[:5])

In [7]:
# dictionary mapping a spreadsheet column name to a RDF property URI
str2property = {'Time':CM.time, 'Agent':CM.agent, 'sequence':CM.sequence, 
                'Type':CM.messageType, 'Level':CM.level, 'Grid':CM.grid,
                'Agent':CM.agent, 'Target':CM.target, 'Alert Messages':CM.alertMessage, 'id':CM.id}

In [8]:
# function mapping prop's value to a RDF property URI
def str2object (msg, prop, value):
    """ convert a string into a object or literal """
    if type(value) in [int, float]:
        return Literal(value)
    elif prop == 'Agent':
        return parse_agent(value)
    elif prop == 'Target':
        if value and " Task " in value:
            # sometimes the Target field has the task
            g.add((task, RDFS.label, Literal(value)))
            return task
        elif not msg['Agent']:
            return parse_agent(value)
        else:
            return parse_target(value, msg=msg)
    else:
        return Literal(value)

### Given a string (e.g., "B CO / 1 - 22") representing a military unit we've not seen before, we parse the string to infer the unit's type (e.g., CM.Company) and its superior units (e.g., Batallion 1; Regiment 22) nd their types, adding information to the RDF graph. The function returns the sgent's URI

In [9]:
def normalize_unit_name(text):
    """ returns a name with a slash between components,
     e.g., SCT_PLT/1/22_IN """
    if not text:
        return ''
    name = text.strip()
    if not name:
        return ''
    if '/' not in name:
        # sometimes there's just the local name
        name = name + ' / 1 / 22 IN'
    name = name.replace('-','/')
    name = name.replace(' ','_')
    if re.search('^\d_\d_CO', name):
        name = name[0] + '/' + name[1:]
    unit = [x.strip('_') for x in name.split('/')]
    name = '/'.join(unit)
    return name

def infer_unit_type_and_parent(name):
    """ returns the type of a unit based on it's name, e.g.,
       SCT_PLT/1/22_IN  => CM.ScoutPlatoon """
    unit = name.split('/')
    unit0 = unit[0]
    parent = '/'.join(unit[1:])
    if 'ENG_CO' in unit0:
        unit_type = CM.EngineeringCompany
    elif 'SCT_PLT' in unit0:
        unit_type = CM.ScoutPlatoon
    elif 'MORTAR_PLT' in unit0:
        unit_type = CM.MortarPlatoon
    elif 'CO' in unit0:
        unit_type = CM.Company
    elif len(unit) > 1 and 'CO' in unit[1]:
        unit_type = CM.Platoon
    else:
        print('Unrecognized unit type:', name)
        unit_type = CM.MilitaryUnit
    return (unit_type, parent)

def parse_target(text, field='target', infer_types=False, msg=None):
    #print('parse target:', text, msg)
    return parse_agent(text, field='target', infer_types=infer_types, msg=msg)

def parse_agent(text, field='agent', infer_types=False, msg=None):
    """returns """
    global instances

    name = normalize_unit_name(text)
    if name in instances:
        # We've seen this before, so just return the instance
        return instances[name]
    
    unit_type, parent = infer_unit_type_and_parent (name)
    print(f"{name} => ({unit_type} {parent})")
      
    id = bnode('UNIT')
    #id = 'UNIT_'+name
    instances[name] = id
    if field == 'agent':
        g.add((id, CM.force, CM.BLUE))
    elif field == 'target' and msg and msg['Agent']:
        g.add((id, CM.force, CM.RED))
    g.add((id, RDF.type, unit_type))
    g.add((id, RDFS.label, Literal(name)))
    
    if not infer_types:
        return id
        
    battalionid = cm + rest
    battalion_type = CM.Battalion

    regimentid = cm + regiment
    if 'IN' in regiment:
        regiment_type = CM.InfantryRegiment
    else:
        regiment_type = CM.Regiment
        
    # add instance data
    if platoonid:
        g.add((platoonid, RDF.type, CM.Platoon))
        g.add((platoonid, CM.unitOf, unitid))
    
    g.add((unitid, RDF.type, unit_type))
    g.add((unitid, CM.unitOf, battalionid))
    
    g.add((battalionid, RDF.type, battalion_type ))
    g.add((battalionid, CM.unitOf, regimentid))
    
    g.add((regimentid, RDF.type, regiment_type))
    
    return instance

def duration(time1, time2):
    # won't work if we cross a day boundry :-( 
    # H+08:12 
    h1,m1  = time1.split('+')[1].split(':')
    h2,m2  = time2.split('+')[1].split(':')
    return str(h2-h1) + ':' + str(m2-m1)
    
def time_to_minutes(time):
    h, m  = [int(x) for x in time.split('+')[1].split(':')]
    return 60*h + m
        

In [10]:
def add_action(msg, msg_id):
    """ add an action to the message"""
    act = bnode('ACT')
    alert = msg["Alert Messages"].lower()
    agent = list(g.objects(msg_id, CM.agent))[0]
    target = list(g.objects(msg_id, CM.target))[0]
    #g.add((msg_id, CM.action, act))
    g.add((act, CM.message, msg_id))
    g.add((act, CM.time, list(g.objects(msg_id, CM.time))[0] ))
    g.add((act, CM.minutes, list(g.objects(msg_id, CM.minutes))[0] ))
    g.add((act, CM.sequence, list(g.objects(msg_id, CM.sequence))[0] ))
    g.add((act, CM.alertMessage, Literal(alert)))
    if 'resupply' in alert:
        g.add((act, RDF.type, CM.Resupply))
        g.add((act, CM.recipient, target))
        if "(ammo)" in alert:
            g.add((act, CM.object, CM.AMMO))
        elif "(fuel)" in alert:
            g.add((act, CM.object, CM.FUEL))
    elif 'earned sa' in alert:
        g.add((act, RDF.type, CM.EarnedSA))
        g.add((act, CM.subject, agent))
        g.add((act, CM.object, target))
    elif 'moving to fight' in alert:
        g.add((act, RDF.type, CM.Move))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
        g.add((act, CM.reason, Literal("attack")))
    elif re.match('attacking.*against', alert):
        g.add((act, RDF.type, CM.Attack))
        g.add((act, CM.subject, agent))
        g.add((act, CM.object, target))
    elif 'fighting' in alert:
        g.add((act, RDF.type, CM.StartFight))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
    elif re.match('attacking.*ended', alert):
        g.add((act, RDF.type, CM.EndFight))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
    elif re.match('receiving.*fire$', alert):
        g.add((act, RDF.type, CM.Attack))
        g.add((act, CM.object, agent))
    elif re.match('receiving.*fire ended', alert):
        g.add((act, RDF.type, CM.EndAttack))
        g.add((act, CM.object, agent))
    elif "not going after opfor" in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.UNABLE ))
        g.add((act, CM.reason, Literal(alert)))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Engage))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif "paused at crossing control point" in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.Status, CM.PAUSE ))
        if "crossing is not traversable" in alert:
            g.add((act, CM.reason, Literal("Crossing not traversable")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.MOVE))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'task' in alert or 'Task' in msg['Alert Messages']:
        # g.add((act, RDF.type, CM.EndAttack))
        g.add((act, RDF.type, CM.TaskReport))
        g.add((act, CM.subject, agent))
        # g.add((act, CM.object, agent))
    elif 'waiting' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.WAIT ))
        if "can't attack without reinforcements" in alert:
            g.add((act, CM.reason, Literal("need reinforcements")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'resume' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.RESUME))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Move))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'planned battle removed' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.END))
        if 'no real targets' in alert:
            g.add((act, CM.reason, Literal("no targets")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'firing has stopped'in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.END ))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.object, agent))
    else:
        print("Unrecognized alert:", alert)
        return None
    return act

In [11]:
# Create the inital RDF graph 
g = Graph()
CM = Namespace('http://purl.org/artiamas/cm/')
g.bind("cm", CM, override=True)
g.bind("owl", OWL, override=True)
g.bind("rdf", RDF, override=True)
instances = {'':CM.NONE} # dict of names to bnodes

simulation = bnode('TASK')
g.add((simulation, RDF.type, CM.Simulation))
task = bnode('TASK')
g.add((task, RDF.type, CM.CrossingTask))
g.add((simulation, CM.task, simulation))

<Graph identifier=N2704f9bfdd684b528e8d60309f1e0d12 (<class 'rdflib.graph.Graph'>)>

In [12]:
# read cm ontology into another graph
gcm = Graph()
gcm.parse("http://purl.org/artiamas/cm", format='ttl')

<Graph identifier=Naea1cd3d2dcb4d5f84e72479112eeb26 (<class 'rdflib.graph.Graph'>)>

### Add message objects to the graph

In [13]:
messages = []
actions = []
for msg in all_data:
    subj = bnode('MSG')
    messages.append(subj)
    g.add((subj, RDF.type, CM.CombatMessage))
    g.add((subj, CM.task, task))
    for prop, obj in msg.items():
        if prop not in str2property:
            print(f"Unrecognized property {prop}")
            continue
        obj = str2object(msg, prop, obj)
        prop = str2property[prop]
        g.add((subj, prop, obj))
    # new properties
    g.add((subj, CM.minutes, Literal(time_to_minutes(msg['Time'])) ))
    act = add_action(msg, subj)
    actions.append(act)
g.add((simulation, CM.firstMessage, messages[0]))
g.add((simulation, CM.lastMessage, messages[-1]))
g.add((simulation, CM.firstAction, actions[0]))
g.add((simulation, CM.lastAction, actions[-1]))
for i in range(len(messages)-1):
     g.add((messages[i], CM.nextMessage, messages[i+1]))
     g.add((actions[i], CM.nextAction, actions[i+1]))

SCT_PLT/1/22_IN => (http://purl.org/artiamas/cm/ScoutPlatoon 1/22_IN)
1/1_CO_(TRK)/1_BN/241_INF_BDE_(TM) => (http://purl.org/artiamas/cm/Platoon 1_CO_(TRK)/1_BN/241_INF_BDE_(TM))
C_CO/1/22_IN => (http://purl.org/artiamas/cm/Company 1/22_IN)
50_ENG_CO_(MRBC)/1/22_IN => (http://purl.org/artiamas/cm/EngineeringCompany 1/22_IN)
A_CO/1/22_IN => (http://purl.org/artiamas/cm/Company 1/22_IN)
D_CO/1/22_IN => (http://purl.org/artiamas/cm/Company 1/22_IN)
MORTAR_PLT/1/22_IN => (http://purl.org/artiamas/cm/MortarPlatoon 1/22_IN)
B_CO/1/22_IN => (http://purl.org/artiamas/cm/Company 1/22_IN)
3/1_CO_(TRK)/1_BN/241_INF_BDE_(TM) => (http://purl.org/artiamas/cm/Platoon 1_CO_(TRK)/1_BN/241_INF_BDE_(TM))
2/1_CO_(TRK)/1_BN/241_INF_BDE_(TM) => (http://purl.org/artiamas/cm/Platoon 1_CO_(TRK)/1_BN/241_INF_BDE_(TM))


### compute deductive closure using owl-rl

In [14]:
# owlrl.DeductiveClosureDeductiveClosure(OWLRL_Extension, rdfs_closure = True, axiomatic_triples = True, datatype_axioms = True).expand(graph), rdfs_closure = True, axiomatic_triples = True, datatype_axioms = True).expand(g)

In [15]:
# g2 = g + gcm
# rdfs = owlrl.RDFSClosure.RDFS_Semantics(g2, True, True, True)
# rdfs.closure()
# rdfs.flush_stored_triples()

In [16]:
#g2 = g + gcm
#superTypeQuery = "select ?X ?ST {?X rdf:type/rdfs:subClassOf* ?ST}"
#qres = g2.query(superTypeQuery)

In [17]:
#for row in qres:
#    print(f"{row.X} a {row.ST}")
#    if row.ST in [OWL.Class, OWL.Restriction, CM.NONE, None] or row.X == CM.MilitaryUnit:
#        #print(f"skipping {row.X} a {row.ST}")
#        pass
#    else:
#        g.add((row.X, RDF.type, row.ST))
#        #print(f"adding {row.X} a {row.ST}")

In [18]:
g.serialize(format='ttl', destination='sim.ttl')

<Graph identifier=N2704f9bfdd684b528e8d60309f1e0d12 (<class 'rdflib.graph.Graph'>)>

In [19]:
print(g.serialize(format='ntriples'))

_:MSG_Diupo <http://purl.org/artiamas/cm/minutes> "245"^^<http://www.w3.org/2001/XMLSchema#integer> .
_:ACT_VpUr9 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://purl.org/artiamas/cm/Move> .
_:MSG_8Kagc <http://purl.org/artiamas/cm/minutes> "367"^^<http://www.w3.org/2001/XMLSchema#integer> .
_:MSG_PekcT <http://purl.org/artiamas/cm/agent> <http://purl.org/artiamas/cm/NONE> .
_:MSG_kW5xW <http://purl.org/artiamas/cm/messageType> "COMBAT" .
_:ACT_M8GaB <http://purl.org/artiamas/cm/alertMessage> "movement is resumed" .
_:ACT_FxNt2 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://purl.org/artiamas/cm/Report> .
_:MSG_Pv6hH <http://purl.org/artiamas/cm/messageType> "COMBAT" .
_:ACT_Ysg2s <http://purl.org/artiamas/cm/message> _:MSG_fHBvW .
_:MSG_i4wT4 <http://purl.org/artiamas/cm/task> _:TASK_W22EP .
_:MSG_GN34G <http://purl.org/artiamas/cm/alertMessage> "not going after OPFOR (out of assigned boundary)" .
_:MSG_NVtBW <http://purl.org/artiamas/cm/minutes> "180"^^<http://www

In [20]:
for name in instances.keys():
    print(name)


SCT_PLT/1/22_IN
1/1_CO_(TRK)/1_BN/241_INF_BDE_(TM)
C_CO/1/22_IN
50_ENG_CO_(MRBC)/1/22_IN
A_CO/1/22_IN
D_CO/1/22_IN
MORTAR_PLT/1/22_IN
B_CO/1/22_IN
3/1_CO_(TRK)/1_BN/241_INF_BDE_(TM)
2/1_CO_(TRK)/1_BN/241_INF_BDE_(TM)


## fin